In [5]:
import dimod
from dwave.system import LeapHybridBQMSampler
from qa_testing import get_api_token

toke = get_api_token()
# -------- Parameters --------
N = 13656          # 13653 = 111 * 123
b = 7              # bits per factor (0..127)

# scale the product term above the tiny 1/2^24 base; try 200..1000
SCALE_MULT = 500.0

# consistency weight: make sure violating m_ij = p_i*q_j is never beneficial
lam = 6000.0

# softly encourage odd factors and break symmetry
mu  = 2.0
eps = 0.001

# -------- Labels & helpers --------
p_bits = [f"p{i}" for i in range(b)]
q_bits = [f"q{j}" for j in range(b)]
m_bits = [[f"m{i}_{j}" for j in range(b)] for i in range(b)]

def w(i, j):
    return 2 ** (i + j)

def bits_to_int(sample, prefix):
    # Cast each sample bit to native int to avoid numpy overflows
    return sum((1 << i) * int(sample[f"{prefix}{i}"]) for i in range(b))

# -------- Build BQM --------
bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)

# (1) Product-matching term: (N - sum w_ij * m_ij)^2 with scaled coefficients
max_i_plus_j = 2 * (b - 1)        # for b=7, = 12
w_max = 2 ** max_i_plus_j         # = 4096
base_scale = 1.0 / (w_max * w_max)  # = 1/2^24
scale = SCALE_MULT * base_scale     # e.g., 500 / 2^24

# Offset from N^2
bqm.offset += scale * (N * N)

# Linear terms in m: -2N * w(i,j)
for i in range(b):
    for j in range(b):
        m = m_bits[i][j]
        bqm.add_variable(m, scale * (-2 * N * w(i, j)))

# Quadratic m-m couplings and diagonal contributions
for i in range(b):
    for j in range(b):
        m1 = m_bits[i][j]
        for ip in range(i, b):
            start_jp = 0 if ip > i else j + 1
            for jp in range(start_jp, b):
                m2 = m_bits[ip][jp]
                coef = scale * (w(i, j) * w(ip, jp))
                if ip == i and jp == j:
                    bqm.add_variable(m1, coef)          # diagonal goes to linear
                else:
                    bqm.add_interaction(m1, m2, coef)   # add, don't read

# (2) Consistency: enforce m_ij = p_i * q_j  (zero iff consistent)
for i in range(b):
    for j in range(b):
        m = m_bits[i][j]
        p = p_bits[i]
        q = q_bits[j]
        bqm.add_interaction(p, q,  lam)         # + lam * (p*q)
        bqm.add_interaction(p, m, -2.0 * lam)   # - 2 lam * (p*m)
        bqm.add_interaction(q, m, -2.0 * lam)   # - 2 lam * (q*m)
        bqm.add_variable(m,  3.0 * lam)         # + 3 lam * m

# (3) Softly encourage odd factors: p0=1, q0=1   (mu*(x-1)^2 = mu*(1 - x))
for var in ("p0", "q0"):
    bqm.offset += mu
    bqm.add_variable(var, -mu)

# (4) Tiny symmetry breaker to prefer p <= q (reduces degeneracy)
for i in range(b):
    bqm.add_variable(f"p{i}",  eps * (1 << i))
    bqm.add_variable(f"q{i}", -eps * (1 << i))

# -------- Solve --------
sampler = LeapHybridBQMSampler()
res = sampler.sample(bqm, label=f"Factor N={N} into p*q (scaled QUBO, S={SCALE_MULT}, lam={lam:g})")

# -------- Inspect results --------
found = False
print("Top candidates:\n")
for idx, r in enumerate(res.data(['sample', 'energy', 'num_occurrences'])):
    s = r.sample
    p_val = bits_to_int(s, "p")
    q_val = bits_to_int(s, "q")
    prod  = int(p_val) * int(q_val)   # ensure native int multiply
    print(f"#{idx+1:02d}  p={p_val:4d}  q={q_val:4d}  p*q={prod:6d}  "
          f"E={r.energy:.6f}  count={r.num_occurrences}")
    if prod == N:
        found = True

print("\nExact factorization found." if found else "\nNo exact factorization in top results.")

Top candidates:

#01  p= 127  q= 127  p*q= 16129  E=-4138.584495  count=1

No exact factorization in top results.


In [ ]:
# Exact factorization as a pure feasibility CQM (no objective)
# Requires: pip install dwave-ocean-sdk
# Auth: `dwave auth login` or env token
import dimod
from dwave.system import LeapHybridCQMSampler

N = 1022117
UPPER = 1100  # adjust as needed

cqm = dimod.CQM()

# Integer factor variables
p = dimod.Integer('p', lower_bound=1, upper_bound=UPPER)
q = dimod.Integer('q', lower_bound=1, upper_bound=UPPER)

# Pure feasibility: product equality + symmetry constraint (optional)
cqm.add_constraint(p * q == N, label="product")


# No objective — keep it a feasibility problem
#cqm.set_objective(0)

# Solve
sampler = LeapHybridCQMSampler()
res = sampler.sample_cqm(cqm, label=f"Factorization N={N} (feasibility)")

# Read best feasible sample
feasible = res.first
s = feasible.sample
p_val = int(s['p'])
q_val = int(s['q'])
print("p =", p_val, " q =", q_val, "  product =", p_val*q_val)
print("feasible:", feasible.is_feasible, "energy:", feasible.energy)

# (Optional) quick primality check
def is_prime(x):
    if x < 2: return False
    if x % 2 == 0: return x == 2
    r = int(x**0.5)
    for d in range(3, r+1, 2):
        if x % d == 0: return False
    return True

print("p is prime?", is_prime(p_val), " q is prime?", is_prime(q_val))


p = 131  q = 101   product = 13231
feasible: True energy: 0.0
p is prime? True  q is prime? True
